In [1]:
# import libraries
from tableau_api_lib import TableauServerConnection
from tableau_api_lib.utils import querying, flatten_dict_column
from urllib import parse
import pandas as pd
import io
import os
from datetime import date  

In [ ]:
# define server configuration
config = {
    'tableau_prod' : {
        'server': '', #https://xxx.online.tableau.com
        'api_version': '3.19', 
        #'username': '',
        #'password':,
        'personal_access_token_name': 'xxx', # name of token in Tableau site
        'personal_access_token_secret': 'xxx', # from Tableau site user settings
        'site_name': 'xxx', # site name and url sometimes the same
        'site_url': 'xxx'
    }
}

# create connection
conn = TableauServerConnection(config_json=config, env='tableau_prod')

#response = conn.sign_in() #response = 200 is good
conn.sign_in()

Datasource ID's

In [ ]:
##This section will give us datasource name and id infor 
# data source info
datasource_df = querying.get_datasources_dataframe(conn)
#print(datasource_df.head(25))
#print(datasource_df.iloc[0])
#print(datasource_df.name.unique())
#datasource_df.dtypes

#create data frame just with name and id
ds_name_id = datasource_df[['name', 'id']]
print(ds_name_id.sort_values(by=['name']))

Workbooks and Sheets

In [ ]:
### Query server

# list of all the views on server
views_df = querying.get_views_dataframe(conn)
views_df = flatten_dict_column(views_df, keys=["name","id"], col_name="workbook")

# workbook we are interested in
comm_sales = views_df[views_df["workbook_name"] == "xxx"] # filter to the workbook of interest

dashboard_view_id = comm_sales[comm_sales['name'] == 'xxx']['id'].values[0] # actual dashboard
filter_view_id = comm_sales[comm_sales['name'] == 'xxx']['id'].values[0] # sheet with list of values for filtering
report_view_id = comm_sales[comm_sales['name'] == 'xxx']['id'].values[0] # raw data table for csv

# data from sheet in Tableau workbook with the names of filter values
filter_data = conn.query_view_data(view_id=filter_view_id)
filter_df = pd.read_csv(io.StringIO(filter_data.content.decode('utf-8')))
filter_list = list(filter_df['xxx']) # name of field used for filter

In [ ]:
### Outside of loop - Params are very customizable

# specify pdf params
pdf_params = {
    "pdf_orientation": "orientation=Portrait",
    "pdf_layout": "type=A4",
    "filter": None
}
# specify csv params
csv_params = {
    "filter": None
}

#folder paths
path = 'xxx' # wherever you want to land the files

In [ ]:
### PDF and CSV generation loop

# global
for i in filter_list:
    date = date.today()

### PDF portion
    pdf_params['filter'] = f'vf_xxx={parse.quote(i)}' # xxx = name of field used for filter
    pdf = conn.query_view_pdf(view_id=dashboard_view_id, parameter_dict=pdf_params)
    pdf_new = open(os.path.join(path,f'{i}.pdf'), 'wb')
    pdf_new.write(pdf.content)    

### CSV Portion
    csv_params['filter'] = f'vf_xxx={parse.quote(i)}' # xxx = name of field used for filter
    report_data = conn.query_view_data(view_id=report_view_id, parameter_dict=csv_params)
    report_df = pd.read_csv(io.StringIO(report_data.content.decode('utf-8')))
    # prep data
    report_data_clean = pd.merge(
        report_df.pivot(index = 'Row', columns = 'Measure Names', values = 'Measure Values'),
        report_df.drop(columns = ['Measure Names', 'Measure Values']).drop_duplicates(),
        on = 'Row').drop(columns = ['xxx', 'xxx']) # whatever fields you want to remove
    cols = ['xxx', 'xxx', 'xxx'] # reorder fields
    report_data_clean = report_data_clean[cols]
    # create to csv
    report_data_clean.to_csv(os.path.join(path,f'{i}.csv'))